## Problems

### Conversion
1.	Is it possible to predict customer conversion (from viewing to purchasing)?
2.	Can we identify “new customers”? If yes then what is the best strategy to work with them?
3.	What are the top features that define customer conversion?
4.	Can we explain and visualize the predictive model for customer conversion on a journey and at a session level?
5.	Is there a pattern and trend across product types regarding the conversion event?

### Personalization
1.	Can we personalize the experience from customer-product interactions?
2.	Does the product catalogue information help in improving customer experience?
3.	Can personalized shopping experience be predictively modeled?
4.	Can we apply transfer learning from other E-commerce data sets to our local data set?

## Preprocessing

In [1]:
from groupby_user_conversion import GroupBy, FeatureSelection

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
g = GroupBy('raw_data.json')
preprocessed_data = g.preprocessing_for_bin_class()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [4]:
# Select features
new_features = FeatureSelection.by_coorelation(preprocessed_data['features'], debug=True)
new_features = FeatureSelection.by_permutation_importance(x=new_features, y=preprocessed_data['label'], debug=True)

Feature "totalRefinements" is highly related to "uniqueRefinements". Remove "totalRefinements"
Feature "uniqueNavigations" is highly related to "uniqueRefinements". Remove "uniqueNavigations"
Feature "totalSearches" is highly related to "events". Remove "totalSearches"
Feature "uniqueViewProducts" is highly related to "events". Remove "uniqueViewProducts"
Feature "totalViewProducts" is highly related to "events". Remove "totalViewProducts"
Feature "has_campaign" is highly related to "source". Remove "has_campaign"
Feature "timeOnSiteSeconds" is highly related to "sessionDuration". Remove "timeOnSiteSeconds"
Feature "totalAddToCarts" is highly related to "uniqueAddToCarts". Remove "totalAddToCarts"
Feature "totalAddToCartRevenue" is highly related to "uniqueAddToCarts". Remove "totalAddToCartRevenue"
28


/home/ec2-user/SageMaker/groupby_user_conversion.py:79: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(x, y)


               score                  feature
feature 11  0.117465  totalViewProductRevenue
feature 10  0.084277         productPriceMean
feature 12  0.066552          sessionDuration
feature 13  0.054529         uniqueAddToCarts
feature 4   0.036189                   events
feature 1   0.026905        totalAddToCartQty
feature 6   0.015775                hourOfDay


## XGBoost Model Training

In [7]:
xgb_model, test_x, test_y = GroupBy.train_xgb_bin_class(new_features, preprocessed_data['label'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Sequential Model Training

In [11]:
preprocessed_data_for_lstm = g.preprocessing_for_sequence_model(num_of_events=30)
lstm_model, test_x, test_y = GroupBy.train_lstm(
    preprocessed_data_for_lstm['features'], preprocessed_data_for_lstm['label'])

Epoch 1/150
27/27 [==============================] - 7s 68ms/step - loss: 0.6810 - recall: 0.8370 - val_loss: 0.6486 - val_recall: 0.9122
Epoch 2/150
27/27 [==============================] - 0s 11ms/step - loss: 0.6348 - recall: 0.9162 - val_loss: 0.5874 - val_recall: 0.9193
Epoch 3/150
27/27 [==============================] - 0s 11ms/step - loss: 0.5680 - recall: 0.9203 - val_loss: 0.5047 - val_recall: 0.9216
Epoch 4/150
27/27 [==============================] - 0s 11ms/step - loss: 0.4826 - recall: 0.9220 - val_loss: 0.4225 - val_recall: 0.9225
Epoch 5/150
27/27 [==============================] - 0s 11ms/step - loss: 0.4019 - recall: 0.9228 - val_loss: 0.3576 - val_recall: 0.9230
Epoch 6/150
27/27 [==============================] - 0s 11ms/step - loss: 0.3362 - recall: 0.9231 - val_loss: 0.3141 - val_recall: 0.9233
Epoch 7/150
27/27 [==============================] - 0s 11ms/step - loss: 0.3007 - recall: 0.9234 - val_loss: 0.2874 - val_recall: 0.9235
Epoch 8/150
27/27 [===============